In [5]:
#자연어 처리를 위해서는 문자를 숫자로 수치화하는 것.
#국소 표현 방법(이산 표현 방법)은 단어에 번호를 부여하는 것.
#분산 표현 방법(연속 표현 방법)은 주변 단어를 참고한다. puppy옆에 cute, lovely가 자주 등장하니까.
#국소는 뉘앙스x, 분산은 뉘앙스o
#BoW는 국소표현, 단어의 빈도수를 카운트해 수치화하는 단어 표현 방법
#LSA는 뉘앙스를 반영하는 연속표현

#BoW: 단어의 출현 빈도에만 집중하는 텍스트 데이터의 수치화 표현 방법

from konlpy.tag import Okt
import re
okt=Okt()

token=re.sub("(\.)","","정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.")
#정규 표현식을 통해 온점을 제거하는 정제 작업
token=okt.morphs(token)
#OKT형태소 분석기를 통해 토큰화 작업을 수행한 뒤에 token에다가 넣습니다. 
word2index={}
bow=[]
for voca in token:
    if voca not in word2index.keys():
        word2index[voca]=len(word2index)
        #token을 읽으면서 word2index에 없는 단어는 추가, 이미 있는 단어는 넘긴다.
        bow.insert(len(word2index)-1,1)
        #bow전체에 전부 기본값 1을 넣어준다. 단어의 개수는 최소 1개 이상이기 때문이다.
    else:
        index=word2index.get(voca)
        #\재등장하는 단어의 인덱스를 받아온다.
        bow[index]=bow[index]+1
        #재등장한 단어는 해당하는 인덱스의 위치에 1을 더해준다.(단어의 개수를 센다.)
print(word2index)
#token을 읽으면서 word2index에 없는 단어는 새로 추가하고 이미 있는 단어는 넘긴다.
bow
#두번째꺼는 해당 인덱스의 글이 몇번 나오는지 확인시켜준다.

{'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}


[1, 2, 1, 1, 2, 1, 1, 1, 1, 1]

In [7]:
#CounterVectorizer클래스로 BoW 만들기
from sklearn.feature_extraction.text import CountVectorizer
corpus=['you know I want your love. because I love you.']
vector=CountVectorizer()
print(vector.fit_transform(corpus).toarray())#코퍼스로부터 각 단어의 빈도수를 기록
print(vector.vocabulary_)

[[1 1 2 1 2 1]]
{'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


In [1]:
#불용어를 제거한 BoW만들기
#불용어는 자연어처리에서 별로 의미를 갖지 않는단어들
#빈도수를 수치화하겠다는 것은 결국 텍스트 내에서 어떤 단어들이 중요한지를 보고싶다.
#
from sklearn.feature_extraction.text import CountVectorizer
text=["Family is not an important thing. It's everything."]
vect=CountVectorizer(stop_words=["the","a","an","is","not"])
print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


In [4]:
from nltk.corpus import stopwords #nltk에서 지원하는 불용서 사용
text=["Family is not an important thing. It's everything."]
sw=stopwords.words("english")
vect=CountVectorizer(stop_words=sw)
print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


In [5]:
#CountVectorizer에서 제공하는 자체 불용어 사용
text=["Family is not an important thing. It's everything."]
vect=CountVectorizer(stop_words="english")
print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)

[[1 1 1]]
{'family': 0, 'important': 1, 'thing': 2}


In [10]:
#문서 단어 행렬 : 서로다른문서들의 BoW들을 결합한 표현방법인 문서단어행렬(DTM)표현방법
#행과 열을 반대로 선택하면 TDM이라고 부르기도 한다.

#TF-IDF : DTM내에 있는 각 단어에 대한 중요도를 계산할 수 있는 가중치
#더 많은 정보를 고려하여 문서를 비교할 수 있다. 그러나 성능이 더 뛰어나지는 않다.
#TF-IDF=Term Frequency Inverse Document Frequency 단어빈도 역문서빈도)

#tf(d,t) : 특정 문서 d에서의 특정단어 t의 등장 횟수
#df(t): 특정단어 t가 등장한 문서의 수 
#idf(d,t):df(t)에 반비례하는 수 = log(n/1+df(t))
#로그를 사용하는 이유는 IDF 값이 기하급수적으로 커지게 되기 때문에 
#불용어와 같이 자주 쓰이는 단어들은 비교적 자주 쓰이지 않는 단어들보다 최소 수십배 자주 등장
#그래서 log를 사용하는 것.

#파이썬으로 TF-IDF 직접 구현하기

import pandas as pd #데이터프레임 사용
from math import log #IDF 계산을 위해
docs=[
    '먹고 싶은 사과',
    '먹고 싶은 바나나',
    '길고 노란 바나나 바나나',
    '저는 과일이 좋아요'
]
vocab=list(set(w for doc in docs for w in doc.split()))
vocab.sort()

N=len(docs)#총 문서의 수
#TF, IDF, TF-IDF값을 구하는 함수 
def tf(t,d):
    return d.count(t)
def idf(t):
    df=0
    for doc in docs:
        df+=t in doc
    return log(N/(df+1))

def tfidf(t,d):
    return tf(t,d)*idf(t)

result=[]
for i in range(N):
    result.append([])
    d=docs[i]
    for j in range(len(vocab)):
        t=vocab[j]
        result[-1].append(tf(t,d))
tf_=pd.DataFrame(result,columns=vocab)
tf_
#정상적으로 DTM이 출력되었다.



,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [9]:
result=[]
for j in range(len(vocab)):
    t=vocab[j]
    result.append(idf(t))

idf_=pd.DataFrame(result,index=vocab,columns=["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [12]:
result=[]
for i in range(N):
    result.append([])
    d=docs[i]
    for j in range(len(vocab)):
        t=vocab[j]
        result[-1].append(tfidf(t,d))
        
tfidf_=pd.DataFrame(result,columns=vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


In [13]:
#사이킷런을 이용한 DTM와 TF-IDF실습
from sklearn.feature_extraction.text import CountVectorizer
corpus=[
    'you know I want your love',
    'I like you',
    'what should I do ',]
vector=CountVectorizer()
print(vector.fit_transform(corpus).toarray())#코퍼스로부터 각 단어의 빈도수를 기록한다.
print(vector.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
tfidfv=TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
